<h1 id=\"tocheading\">Table of Contents</h1>
    <div id=\"toc\"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// needed to generate the Table of contents 
// taken from github.com/kmahelona/ipython_notebook_goodies

# Modeling

## Fit Models with Training Data Set

## Review Model Outcomes — Iterate over additional models as needed.

## Identify the Final Model

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler

In [24]:
#Load data and scaler model

X_train_scaled = pd.read_csv("../data/X_train_scaled.csv").to_numpy()
y_train = pd.read_csv("../data/y_train.csv").to_numpy().ravel()
X_test_scaled = pd.read_csv("../data/X_test_scaled.csv").to_numpy()
y_test = pd.read_csv("../data/y_test.csv").to_numpy().ravel()
feature_list = pd.read_csv("../data/feature_list.csv")
feature_list = pd.Index(list(feature_list["0"]))

scaler = joblib.load("../data/scaler.pkl")

In [25]:
#initialize dict to collect model performance data
models_comparison = {'Model': [],
        'Explained Variance': [],
        'Mean Absolute Error': [],
        'Features Dropped': []
        }

In [65]:
from timeit import default_timer as timer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

# from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh ???

names = ["Nearest Neighbors", "Linear SVM", "Decision Tree", "Random Forest", "AdaBoost",
         "Gradient Boosting"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GradientBoostingClassifier()]

all_params = { 'dictA': {'key_1': 'value_1'},
                'dictB': {'key_2': 'value_2'}}

In [112]:
# Nearest Neighbors
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':np.arange(1,101,2)}

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn,param_grid,cv=5)
start = timer()
knn_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))

345.662 seconds elapsed.
Best Score:0.773882901822539
Best Parameters: {'n_neighbors': 83}


In [91]:
# Decision Tree
param_grid = {'criterion':['gini', 'entropy'],
              'max_depth':np.arange(1,11,2),
              }
tree = DecisionTreeClassifier()
tree_cv= GridSearchCV(tree,param_grid,cv=5)
start = timer()
tree_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(tree_cv.best_score_))
print("Best Parameters: " + str(tree_cv.best_params_))

5.032 seconds elapsed.
Best Score:0.7610855342864066
Best Parameters: {'criterion': 'gini', 'max_depth': 7}


In [111]:
# Random Forest
param_grid = {'n_estimators':np.arange(50,201,50),
              'criterion':['gini', 'entropy'],
              'max_depth':np.arange(1,11,2),
              }
rfc = RandomForestClassifier()
rfc_cv= GridSearchCV(rfc,param_grid,cv=5)
start = timer()
rfc_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(rfc_cv.best_score_))
print("Best Parameters: " + str(rfc_cv.best_params_))

437.482 seconds elapsed.
Best Score:0.7851760764416167
Best Parameters: {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 150}


In [110]:
# Suppor Vector Classifier - Linear
param_grid = {'C' : np.arange(1,100,10),
              'kernel': ['linear'],
              'gamma' : ['sacle','auto'],
              }
svc = SVC()
svc_cv = GridSearchCV(svc,param_grid,cv=5)
start = timer()
svc_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(svc_cv.best_score_))
print("Best Parameters: " + str(svc_cv.best_params_))

/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/svm/_base.py", line 205, in fit
    raise ValueError(
ValueError: When 'gamma' is a string, it should be either 'scale' or 'auto'. Got 'sacle' instead.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan

/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/svm/_base.py", line 205, in fit
    raise ValueError(
ValueError: When 'gamma' is a string, it should be either 'scale' or 'auto'. Got 'sacle' instead.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan

/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/svm/_base.py", line 205, in fit
    raise ValueError(
ValueError: When 'gamma' is a string, it should be either 'scale' or 'auto'. Got 'sacle' instead.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan

/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/svm/_base.py", line 205, in fit
    raise ValueError(
ValueError: When 'gamma' is a string, it should be either 'scale' or 'auto'. Got 'sacle' instead.

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan

302.803 seconds elapsed.
Best Score:0.778923904284249
Best Parameters: {'C': 51, 'gamma': 'auto', 'kernel': 'linear'}


In [109]:
# Suppor Vector Classifier - RBF
param_grid = {'C' : np.arange(1,100,10),
              'kernel': ['rbf'],
              'gamma' : ['scale','auto'],
              }
svc = SVC()
svc_cv = GridSearchCV(svc,param_grid,cv=5)
start = timer()
svc_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(svc_cv.best_score_))
print("Best Parameters: " + str(svc_cv.best_params_))

1303.876 seconds elapsed.
Best Score:0.7826071784581655
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
# AdaBoost
param_grid = {'n_estimators': [10,20,30,40,50],
              'learning_rate': [0.05, 0.1, 0.25, 0.5, 0.75, 1]
              }
ada = AdaBoostClassifier()
ada_cv= GridSearchCV(ada,param_grid,cv=5)
start = timer()
ada_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(ada_cv.best_score_))
print("Best Parameters: " + str(ada_cv.best_params_))

In [114]:
# GradientBoosting
param_grid = {'n_estimators': [10,20,30,40,50],
              'learning_rate': [0.05, 0.1, 0.25, 0.5, 0.75, 1],
              'max_features': [2,3,4,5,6,7,8,9],
              'max_depth' : [2,3,4,5,6]
              }
gbc = GradientBoostingClassifier()
gbc_cv= GridSearchCV(gbc,param_grid,cv=5)
start = timer()
gbc_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(gbc_cv.best_score_))
print("Best Parameters: " + str(gbc_cv.best_params_))

KeyboardInterrupt: 

In [104]:
# XGBoost
param_grid = {
    'learning_rate': np.arange(0.01, 1.0,0.5),
    'n_estimators': np.arange(100, 1000,500),
    'max_depth': np.arange(3,10,3),
    'gamma': np.arange(0, 5,2)}

xgb = XGBClassifier()
xgb_cv= GridSearchCV(xgb,param_grid,cv=3)
start = timer()
xgb_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')

#print("Best Score:" + str(xgb_cv.best_score_))
#print("Best Parameters: " + str(xgb_cv.best_params_))

1324.792 seconds elapsed.


In [105]:
print("Best Score:" + str(xgb_cv.best_score_))
print("Best Parameters: " + str(xgb_cv.best_params_))

Best Score:0.7808152779797393
Best Parameters: {'gamma': 4, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 600}


## Bayesian Optimization

```python
def hyperparam_loss(param_x, param_y):
    
    # 1. Define machine learning model using param_x, param_y as hyper parameters
    # 2. Train the model
    # 3. Calculate loss on cross-validation set
    
    return loss
```

In [91]:
# Decision Tree
param_grid = {'criterion':['gini', 'entropy'],
              'max_depth':np.arange(1,11,2),
              }
tree = DecisionTreeClassifier()
tree_cv= GridSearchCV(tree,param_grid,cv=5)
start = timer()
tree_cv.fit(X_train_scaled,y_train)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')
print("Best Score:" + str(tree_cv.best_score_))
print("Best Parameters: " + str(tree_cv.best_params_))

5.032 seconds elapsed.
Best Score:0.7610855342864066
Best Parameters: {'criterion': 'gini', 'max_depth': 7}


In [ ]:
# Let's rewrite this in Bayesian Optimization terms

#DecisionTree

#def tree__hyper_param():
    

In [ ]:
# from bayes_opt import BayesianOptimization

# name = names[0]
# model = classifiers[0]

# params_boundaries = {'n_neighbors':(3,11)}

# def kNN_hyper_param(n_neighbors):
#     clf = KNeighborsClassifier(n_neighbors)
       
#     return np.mean(cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring='accuracy'))

# optimizer = BayesianOptimization(f=kNN_hyper_param, pbounds=params_boundaries, random_state = 33)
# optimizer.maximize(n_iter=5, init_points=2)

In [98]:
# XGBoost
from xgboost import XGBClassifier

pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10),
    'subsample': (1.0, 1.0),  # Change for big datasets
    'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma)
    return np.mean(cross_val_score(clf, X_train_scaled, y_train, cv=3, scoring='accuracy'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

In [99]:
start = timer()
optimizer.maximize(n_iter=10, init_points=2)
end = timer()
print(str(round(end - start,3)) + ' seconds elapsed.')

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        |  0.7763   |  1.0      |  3.602    |  0.01011  |  5.116    |  232.1    |  1.0      |
|  2        |  0.7753   |  1.0      |  1.728    |  0.4028   |  6.772    |  477.3    |  1.0      |


/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_num

|  3        |  0.7765   |  1.0      |  3.284    |  0.7515   |  4.453    |  233.0    |  1.0      |


/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_num

|  4        |  0.7768   |  1.0      |  1.313    |  0.515    |  3.339    |  239.4    |  1.0      |


...

|  10       |  0.7773   |  1.0      |  4.801    |  0.5464   |  4.774    |  238.3    |  1.0      |


/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_num

|  11       |  0.7755   |  1.0      |  4.113    |  0.8911   |  3.16     |  228.3    |  1.0      |


/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:519: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx
/home/andrea/anaconda3/envs/Jet_clean/lib/python3.8/site-packages/scipy/optimize/_num

|  12       |  0.7743   |  1.0      |  1.631    |  0.2424   |  7.543    |  228.8    |  1.0      |
279.986 seconds elapsed.


In [ ]:
optimize.max